In [1]:
! pip uninstall -y transformers
! pip install --quiet transformers==4.48.3

Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3


In [3]:
import os
import torch
import pandas as pd
import transformers
from google.colab import drive
from transformers import AutoModelForImageClassification, ViTHybridForImageClassification
from datasets import load_dataset

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
print(transformers.__version__)

4.48.3


In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def get_model_param_info(model_names, output_file):
    model_data = []

    for model_name in model_names:
        print(f"\n🔍 Model: {model_name}")

        # Load model based on type
        if "hybrid" in model_name:
            model = ViTHybridForImageClassification.from_pretrained(model_name, num_labels=8, ignore_mismatched_sizes=True)
        else:
            model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=8, ignore_mismatched_sizes=True)

        # Identify trainable parameters
        for name, param in model.named_parameters():
            param.requires_grad = any(head in name for head in ["head", "classifier"])

        # Count parameters
        num_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"Total Parameters: {num_params:,} | Trainable: {trainable_params:,}")

        # Append to list
        model_data.append({
            "Model": model_name,
            "Total Parameters": num_params,
            "Trainable Parameters": trainable_params,
        })

    # Convert to DataFrame and save to Excel
    df_models = pd.DataFrame(model_data)
    return df_models

def get_dataset_info(dataset_path):
    dataset = load_dataset(dataset_path)

    # Get label names from the dataset metadata
    label_names = dataset["train"].features["label"].names  # Assuming all splits have the same labels

    class_data = []

    for split in dataset.keys():
        df = dataset[split].to_pandas()
        class_counts = df["label"].value_counts().reset_index()
        class_counts.columns = ["Class Index", "Count"]
        class_counts["Class"] = class_counts["Class Index"].map(lambda x: label_names[x])  # Map index to label
        class_counts["Split"] = split
        class_counts = class_counts[["Class", "Class Index", "Count", "Split"]]  # Reorder columns
        class_data.append(class_counts)

    # Merge all splits into one DataFrame
    df_dataset = pd.concat(class_data, ignore_index=True)
    return df_dataset

In [13]:
model_list = [
  "facebook/convnextv2-base-1k-224",
  "google/vit-hybrid-base-bit-384",
  "google/vit-base-patch16-224",
  "microsoft/swin-base-patch4-window7-224",
  "facebook/deit-base-patch16-224",
  "facebook/dinov2-base",
  "timm/vit_small_patch16_224.augreg_in21k",
  "microsoft/swin-tiny-patch4-window7-224",
  "facebook/deit-small-patch16-224",
  "facebook/convnextv2-tiny-1k-224",
  "apple/mobilevit-small",
  "timm/mobilevitv2_150.cvnets_in22k_ft_in1k",
  "google/efficientnet-b2",
  "timm/efficientvit_b1.r224_in1k",
  "timm/efficientvit_m4.r224_in1k",
  "timm/efficientformerv2_s2.snap_dist_in1k",
  "timm/efficientformer_l1.snap_dist_in1k"
]


dataset_path = 'cvmil/rice-leaf-disease-augmented-v4'
output_file = "/content/drive/Shareddrives/CS198-Drones/[TEST] Models/model_and_dataset_info.xlsx"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Get model and dataset info
df_models = get_model_param_info(model_list, output_file)
df_dataset = get_dataset_info(dataset_path)

# Save both tables to the same Excel file
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_models.to_excel(writer, sheet_name="Model Parameters", index=False)
    df_dataset.to_excel(writer, sheet_name="Dataset Info", index=False)

print(f"\n✅ Results saved to {output_file}")


🔍 Model: facebook/convnextv2-base-1k-224


Some weights of ConvNextV2ForImageClassification were not initialized from the model checkpoint at facebook/convnextv2-base-1k-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([8, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters: 87,701,000 | Trainable: 8,200

🔍 Model: google/vit-hybrid-base-bit-384


Some weights of ViTHybridForImageClassification were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters: 98,188,104 | Trainable: 6,152

🔍 Model: google/vit-base-patch16-224


...

Total Parameters: 8,422,308 | Trainable: 3,848

🔍 Model: timm/efficientformerv2_s2.snap_dist_in1k


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/efficientformerv2_s2.snap_dist_in1k and are newly initialized because the shapes did not match:
- head.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- head.weight: found shape torch.Size([1000, 288]) in the checkpoint and torch.Size([8, 288]) in the model instantiated
- head_dist.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- head_dist.weight: found shape torch.Size([1000, 288]) in the checkpoint and torch.Size([8, 288]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters: 12,136,736 | Trainable: 5,776

🔍 Model: timm/efficientformer_l1.snap_dist_in1k


Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/efficientformer_l1.snap_dist_in1k and are newly initialized because the shapes did not match:
- head.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- head.weight: found shape torch.Size([1000, 448]) in the checkpoint and torch.Size([8, 448]) in the model instantiated
- head_dist.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- head_dist.weight: found shape torch.Size([1000, 448]) in the checkpoint and torch.Size([8, 448]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters: 11,399,112 | Trainable: 7,184

✅ Results saved to /content/drive/Shareddrives/CS198-Drones/[TEST] Models/model_and_dataset_info.xlsx
